In [ ]:
!git clone --recursive https://github.com/Microsoft/LightGBM
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import DataFrame, concat
!pip install import_ipynb
import import_ipynb
from google.colab import drive
drive.mount('/content/drive')
from datetime import date
import lightgbm as lgbm
from lightgbm import LGBMRegressor
import time
import statsmodels.api as sm
from sklearn import linear_model, metrics
from sklearn.model_selection import train_test_split
from tqdm import tnrange, tqdm_notebook
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.regression.quantile_regression import QuantReg
from sklearn.ensemble import GradientBoostingRegressor
import os

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=913433cc3e67ac54ccc31d6e3e9a23990eb2fe4158583c60362b307b0cdbf9c4
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
Mounted at /content/drive


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
import math
from math import radians

In [4]:
#### 파라미터 ####

# 하루의 틱
ticks = 48

# 예측에 사용할 일수
days = 3                # 이 부분을 바꿀 수 있다.
n_days = ticks*days

# 미래 예측할 일수
future_days = 2
future_window = ticks * future_days

### 모든변수
# ['Hour', 'Minute', 'Day', 'WS', 'Time', 'DHI','DNI','RH','T','TARGET']

# 사용할 변수
what_to_left = ['Time', 'WS', 'DHI','DNI','RH','T','TARGET', 'Day']

n_features = len(what_to_left)
n_obs = n_days * n_features # 7일 예측 기준으로 7(일 수) * 48(틱 수) * 6(변수 개수)

In [5]:
# 데이터 불러오기 

raw_data = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/train/train.csv')
submission = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/sample_submission.csv')
submission.shape

(7776, 10)

In [6]:
def drop_clms(dataset):
  dataset['Time'] = dataset['Hour'] + dataset['Minute']*(0.5/30)
  dataset['Date'] = dataset['Day']%365
  dataset.drop('Day', axis=1, inplace=True)
  
  temp = list()
  for i in range(0, len(dataset), 48):
    temp += [24-(list(dataset.DHI[i:i+48]).count(0)*0.5)]*48 

  dataset["SH"] = temp

  return dataset

def cos_time(dataset):
  dataset['sin_time'] = np.sin(2*np.pi*dataset.Time/24)
  dataset['cos_time'] = np.cos(2*np.pi*dataset.Time/24)
  return dataset

In [7]:
def get_yday(when):
  HP = date(2020,1,1)
  results = (when-HP).days
  return results

In [8]:
def 절기24(Date):
  target = 0
  for i in list24:
    if Date < i:
      target = list24.index(i) - 1
      break
  if Date < 5:
    target = 23
  return target

In [9]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 series_to_supervised 함수

def train_to_supervised(train, n_in):

    clmns = list(train.columns)

    # 타켓 칼럼의 이름을 여기에 입력
    target = ['TARGET']

    # 클래스 변수로 전환(encoding)할 칼럼을 여기에 입력
    class_lst = ['season']

    # 기타 칼럼은 전과 같이 들어갈 것.
    scaled_lst = list(set(clmns) - set(class_lst))

    scaled_df = train[scaled_lst]
    class_df = train[class_lst]
    target_df = train[target]

    # 미래 몇 번째 항목을 가져올 것인가
    future = [48, 96]

    ### 만약에 스케일링을 하고 싶다면 ###
    # scaled_df 데이터 프레임만 스케일링 하고, 절기랑 TARGET 데이터는 그냥 두면 된다.

    # 스케일링 해도 되고, 안해도 되는 기존에 썻던 변수들 전처리
    cols, names = list(), list()
    n_vars = 1 if type(scaled_df) is list else scaled_df.shape[1]
    n_vars2 = 1 if type(target_df) is list else target_df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(scaled_df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in scaled_df.columns]

    # class라 encoding 필요한 절기 끝에 하나만 붙여놓기
    cols.append(class_df)
    names += class_lst

    # 48과 96 후의 타겟 데이터 2개 붙이기.
    # forecast sequence (t, t+1, ... t+n)
    for i in future:
        cols.append(target_df.shift(-i))
        if i == 0:
            names += [('TARGET%d(t)' % (j+1)) for j in range(n_vars2)]
        else:
            names += [('TARGET%d(t+%d)' % (j+1, i)) for j in range(n_vars2)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg

In [10]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 series_to_supervised 함수

def test_to_supervised(train, n_in):

    clmns = list(train.columns)

    # 타켓 칼럼의 이름을 여기에 입력
    target = ['TARGET']

    # 클래스 변수로 전환(encoding)할 칼럼을 여기에 입력
    class_lst = ['season']

    # 기타 칼럼은 전과 같이 들어갈 것.
    scaled_lst = list(set(clmns) - set(class_lst))

    scaled_df = train[scaled_lst]
    class_df = train[class_lst]

    ### 만약에 스케일링을 하고 싶다면 ###
    # testset의 스케일링은 구조가 상당히 까다로우므로....... 일단 나중에 하기로 함.

    # 스케일링 해도 되고, 안해도 되는 기존에 썻던 변수들 전처리
    cols, names = list(), list()
    n_vars = 1 if type(scaled_df) is list else scaled_df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(scaled_df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in scaled_df.columns]

    # class라 encoding 필요한 절기 끝에 하나만 붙여놓기
    cols.append(class_df)
    names += class_lst
    
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg

In [11]:
소한 = get_yday(date(2020,1,6))
대한 = get_yday(date(2020,1,20))
입춘 = get_yday(date(2020,2,4))
우수 = get_yday(date(2020,2,19))
경칩 = get_yday(date(2020,3,6))
춘분 = get_yday(date(2020,3,21))
청명 = get_yday(date(2020,4,5))
곡우 = get_yday(date(2020,4,20))
입하 = get_yday(date(2020,5,6))
소만 = get_yday(date(2020,5,21))
망종 = get_yday(date(2020,6,6))
하지 = get_yday(date(2020,6,22))
소서 = get_yday(date(2020,7,7))
대서 = get_yday(date(2020,7,23))
입추 = get_yday(date(2020,8,8))
처서 = get_yday(date(2020,8,23))
백로 = get_yday(date(2020,9,8))
추분 = get_yday(date(2020,9,23))
한로 = get_yday(date(2020,10,8))
상강 = get_yday(date(2020,10,24))
입동 = get_yday(date(2020,11,8))
소설 = get_yday(date(2020,11,22))
대설 = get_yday(date(2020,12,7))
동지 = get_yday(date(2020,12,22))

list24 = [소한, 대한, 입춘, 우수, 경칩, 춘분, 청명, 곡우, 입하, 소만, 망종, 하지, 소서, 대서, 입추, 처서, 백로, 추분, 한로, 상강, 입동, 소설, 대설, 동지]

In [12]:
m = 60

일출 = [7+47/m, 7+44/m, 7+34/m, 7+18/m, 6+56/m, 6+35/m, 6+12/m, 5+51/m, 5+33/m, 5+19/m, 5+11/m, 5+11/m, 5+17/m, 5+27/m, 5+41/m, 5+54/m, 6+8/m, 6+20/m, 6+33/m, 6+47/m, 7+3/m, 7+20/m, 7+33/m, 7+45/m]
일몰 = [17+28/m, 17+42/m, 17+58/m, 18+15/m, 18+30/m, 18+44/m, 18+58/m, 19+12/m, 19+25/m, 19+39/m, 19+50/m, 19+57/m, 19+56/m, 19+50/m, 19+35/m, 19+15/m, 18+51/m, 18+28/m, 18+6/m, 17+45/m, 17+28/m, 17+17/m, 17+13/m, 17+17/m]
경사각 = [32.92, 36.83, 40.75, 44.67, 48.58, 52.5, 56.42, 60.33, 64.25, 68.16, 72.01, 76, 72.1, 68.16, 64.25, 60.33, 56.42, 52.5, 48.58, 44.67, 40.75, 36.83, 32.92, 29]
남중 = [12+36/m, 12+41/m, 12+44/m, 12+44/m, 12+42/m, 12+38/m, 12+34/m, 12+29/m, 12+27/m, 12+27/m, 12+29/m, 12+32/m, 12+35/m, 12+37/m, 12+36/m, 12+33/m, 12+29/m, 12+23/m, 12+18/m, 12+15/m, 12+14/m, 12+16/m, 12+22/m, 12+29/m]
print(len(일출), len(일몰), len(경사각), len(남중))

24 24 24 24


In [13]:
def HRA(DHI, DNI, season, hour):
  # 위도(latitude) 기준을 일단 임의로 대전으로 설정 (위도 36.19~36.2도)
  latitude = radians(36.2)
  season = int(season)
  # 절기별 대한민국의 경사각
  tilt = radians(경사각[season])

  # 절기별 대한민국 대전의 태양 남중시각
  hra = radians(15*(hour - 남중[season]))

  # 구하려는 알파
  elevation = np.arcsin(np.sin(tilt) * np.sin(latitude) + np.cos(tilt) * np.cos(latitude) * np.cos(hra))

  # 천정각(Zenith Angle)은 90 - 알파
  zenith = radians(90) - elevation

  # GHI는 DHI + DNI * cos(천정각)
  ghi = DHI + DNI *np.cos(zenith)

  return ghi

In [14]:
def encoding(data):
  season_list = [i for i in range(0,24)]
  for k in season_list:
    data['Season_' +f'{k}'] = data['season'] == k
  return data*1

In [15]:
def not_minus(dataset):
  for i in range(0, len(dataset.index)):
    for j in range(0, len(dataset.columns)):
      K = dataset.iloc[i,j]
      if K < 0:
        dataset.iloc[i,j] = 0
  return dataset

In [16]:
# 차원축소
# 원하는 변수의 숫자만큼 남긴다.
# dimension 변수에 원하는 변수의 숫자를 적어준다.

def alzip(dataset, dimension):
    X = dataset.iloc[:,:-1]
    y = dataset.iloc[:,-1:]
    pca = PCA(n_components= dimension)
    pca.fit(X)
    pica_X = pd.DataFrame(pca.transform(X), index = X.index)
    dataset2 = pd.concat([pica_X, y], axis = 1)
    return dataset2

In [17]:
dataset = raw_data.copy()
dataset2 = drop_clms(dataset)
dataset2.drop(['Hour', 'Minute'], axis =1, inplace=True)
dataset2

,DHI,DNI,WS,RH,T,TARGET,Time,Date,SH
0,0,0,1.5,69.08,-12,0.0,0.0,0,9.0
1,0,0,1.5,69.06,-12,0.0,0.5,0,9.0
2,0,0,1.6,71.78,-12,0.0,1.0,0,9.0
3,0,0,1.6,71.75,-12,0.0,1.5,0,9.0
4,0,0,1.6,75.20,-12,0.0,2.0,0,9.0
...,...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,0.0,21.5,364,9.0
52556,0,0,2.4,66.79,-4,0.0,22.0,364,9.0
52557,0,0,2.2,66.78,-4,0.0,22.5,364,9.0
52558,0,0,2.1,67.72,-4,0.0,23.0,364,9.0


In [18]:
dataset2['season'] = dataset2.apply(lambda x: 절기24(x['Date']), axis = 1)
dataset3 = cos_time(dataset2)
dataset4 = dataset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]
# dataset5 = dataset4.iloc[n_days:,:-1]
# goals = pd.DataFrame(dataset4.Goal, index = dataset4.index)
dataset4

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time
0,0,0,1.5,69.08,-12,0.0,9.0,0.000000,1.000000,23,0.0
1,0,0,1.5,69.06,-12,0.0,9.0,0.130526,0.991445,23,0.5
2,0,0,1.6,71.78,-12,0.0,9.0,0.258819,0.965926,23,1.0
3,0,0,1.6,71.75,-12,0.0,9.0,0.382683,0.923880,23,1.5
4,0,0,1.6,75.20,-12,0.0,9.0,0.500000,0.866025,23,2.0
...,...,...,...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,0.0,9.0,-0.608761,0.793353,0,21.5
52556,0,0,2.4,66.79,-4,0.0,9.0,-0.500000,0.866025,0,22.0
52557,0,0,2.2,66.78,-4,0.0,9.0,-0.382683,0.923880,0,22.5
52558,0,0,2.1,67.72,-4,0.0,9.0,-0.258819,0.965926,0,23.0


In [19]:
dataset5= dataset4.copy()

dataset5['GHI'] = dataset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)

In [20]:
dataset5.iloc[50:51,:]

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time,GHI
50,0,0,1.6,88.11,-11,0.0,9.0,0.258819,0.965926,23,1.0,0.0


In [21]:
dataset6 = dataset5[['DHI', 'DNI', 'WS', 'RH', 'T','SH', 'sin_time', 'cos_time', 'GHI', 'season', 'TARGET']]


dataset6.head(10)

,DHI,DNI,WS,RH,T,SH,sin_time,cos_time,GHI,TARGET
0,0,0,1.5,69.08,-12,9.0,0.000000,1.000000,0.0,0.0
1,0,0,1.5,69.06,-12,9.0,0.130526,0.991445,0.0,0.0
2,0,0,1.6,71.78,-12,9.0,0.258819,0.965926,0.0,0.0
3,0,0,1.6,71.75,-12,9.0,0.382683,0.923880,0.0,0.0
4,0,0,1.6,75.20,-12,9.0,0.500000,0.866025,0.0,0.0
5,0,0,1.5,69.29,-11,9.0,0.608761,0.793353,0.0,0.0
6,0,0,1.5,72.56,-11,9.0,0.707107,0.707107,0.0,0.0
7,0,0,1.4,72.55,-11,9.0,0.793353,0.608761,0.0,0.0
8,0,0,1.3,74.62,-11,9.0,0.866025,0.500000,0.0,0.0
9,0,0,1.3,74.61,-11,9.0,0.923880,0.382683,0.0,0.0


## dataset6 가지고 동일하게 to_supervised 함수를 써서 나누면 됩니다.
### 나머지 변수 합치기, 인코딩은 유진이가 해줄 것...

In [21]:
dataset7 = train_to_supervised(dataset6, n_days)
dataset7

,sin_time(t-144),TARGET(t-144),DNI(t-144),T(t-144),SH(t-144),DHI(t-144),RH(t-144),WS(t-144),cos_time(t-144),GHI(t-144),sin_time(t-143),TARGET(t-143),DNI(t-143),T(t-143),SH(t-143),DHI(t-143),RH(t-143),WS(t-143),cos_time(t-143),GHI(t-143),sin_time(t-142),TARGET(t-142),DNI(t-142),T(t-142),SH(t-142),DHI(t-142),RH(t-142),WS(t-142),cos_time(t-142),GHI(t-142),sin_time(t-141),TARGET(t-141),DNI(t-141),T(t-141),SH(t-141),DHI(t-141),RH(t-141),WS(t-141),cos_time(t-141),GHI(t-141),...,T(t-4),SH(t-4),DHI(t-4),RH(t-4),WS(t-4),cos_time(t-4),GHI(t-4),sin_time(t-3),TARGET(t-3),DNI(t-3),T(t-3),SH(t-3),DHI(t-3),RH(t-3),WS(t-3),cos_time(t-3),GHI(t-3),sin_time(t-2),TARGET(t-2),DNI(t-2),T(t-2),SH(t-2),DHI(t-2),RH(t-2),WS(t-2),cos_time(t-2),GHI(t-2),sin_time(t-1),TARGET(t-1),DNI(t-1),T(t-1),SH(t-1),DHI(t-1),RH(t-1),WS(t-1),cos_time(t-1),GHI(t-1),season,TARGET1(t+48),TARGET1(t+96)
144,0.000000,0.0,0.0,-12.0,9.0,0.0,69.08,1.5,1.000000,0.0,0.130526,0.0,0.0,-12.0,9.0,0.0,69.06,1.5,0.991445,0.0,0.258819,0.0,0.0,-12.0,9.0,0.0,71.78,1.6,0.965926,0.0,0.382683,0.0,0.0,-12.0,9.0,0.0,71.75,1.6,0.923880,0.0,...,-13.0,9.0,0.0,79.09,0.4,0.866025,0.0,-0.382683,0.0,0.0,-13.0,9.0,0.0,79.08,0.7,0.923880,0.0,-0.258819,0.0,0.0,-13.0,9.0,0.0,81.09,1.1,0.965926,0.0,-0.130526,0.0,0.0,-13.0,9.0,0.0,81.08,1.4,0.991445,0.0,23,0.0,0.0
145,0.130526,0.0,0.0,-12.0,9.0,0.0,69.06,1.5,0.991445,0.0,0.258819,0.0,0.0,-12.0,9.0,0.0,71.78,1.6,0.965926,0.0,0.382683,0.0,0.0,-12.0,9.0,0.0,71.75,1.6,0.923880,0.0,0.500000,0.0,0.0,-12.0,9.0,0.0,75.20,1.6,0.866025,0.0,...,-13.0,9.0,0.0,79.08,0.7,0.923880,0.0,-0.258819,0.0,0.0,-13.0,9.0,0.0,81.09,1.1,0.965926,0.0,-0.130526,0.0,0.0,-13.0,9.0,0.0,81.08,1.4,0.991445,0.0,0.000000,0.0,0.0,-14.0,9.0,0.0,77.76,1.8,1.000000,0.0,23,0.0,0.0
146,0.258819,0.0,0.0,-12.0,9.0,0.0,71.78,1.6,0.965926,0.0,0.382683,0.0,0.0,-12.0,9.0,0.0,71.75,1.6,0.923880,0.0,0.500000,0.0,0.0,-12.0,9.0,0.0,75.20,1.6,0.866025,0.0,0.608761,0.0,0.0,-11.0,9.0,0.0,69.29,1.5,0.793353,0.0,...,-13.0,9.0,0.0,81.09,1.1,0.965926,0.0,-0.130526,0.0,0.0,-13.0,9.0,0.0,81.08,1.4,0.991445,0.0,0.000000,0.0,0.0,-14.0,9.0,0.0,77.76,1.8,1.000000,0.0,0.130526,0.0,0.0,-13.0,9.0,0.0,77.77,2.0,0.991445,0.0,23,0.0,0.0
147,0.382683,0.0,0.0,-12.0,9.0,0.0,71.75,1.6,0.923880,0.0,0.500000,0.0,0.0,-12.0,9.0,0.0,75.20,1.6,0.866025,0.0,0.608761,0.0,0.0,-11.0,9.0,0.0,69.29,1.5,0.793353,0.0,0.707107,0.0,0.0,-11.0,9.0,0.0,72.56,1.5,0.707107,0.0,...,-13.0,9.0,0.0,81.08,1.4,0.991445,0.0,0.000000,0.0,0.0,-14.0,9.0,0.0,77.76,1.8,1.000000,0.0,0.130526,0.0,0.0,-13.0,9.0,0.0,77.77,2.0,0.991445,0.0,0.258819,0.0,0.0,-13.0,9.0,0.0,77.69,2.2,0.965926,0.0,23,0.0,0.0
148,0.500000,0.0,0.0,-12.0,9.0,0.0,75.20,1.6,0.866025,0.0,0.608761,0.0,0.0,-11.0,9.0,0.0,69.29,1.5,0.793353,0.0,0.707107,0.0,0.0,-11.0,9.0,0.0,72.56,1.5,0.707107,0.0,0.793353,0.0,0.0,-11.0,9.0,0.0,72.55,1.4,0.608761,0.0,...,-14.0,9.0,0.0,77.76,1.8,1.000000,0.0,0.130526,0.0,0.0,-13.0,9.0,0.0,77.77,2.0,0.991445,0.0,0.258819,0.0,0.0,-13.0,9.0,0.0,77.69,2.2,0.965926,0.0,0.382683,0.0,0.0,-13.0,9.0,0.0,71.57,2.2,0.923880,0.0,23,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52459,-0.608761,0.0,0.0,-8.0,9.0,0.0,84.85,1.0,0.793353,0.0,-0.500000,0.0,0.0,-8.0,9.0,0.0,82.26,1.2,0.866025,0.0,-0.382683,0.0,0.0,-8.0,9.0,0.0,82.25,1.3,0.923880,0.0,-0.258819,0.0,0.0,-8.0,9.0,0.0,81.14,1.5,0.965926,0.0,...,0.0,9.0,0.0,51.77,1.9,0.382683,0.0,-0.866025,0.0,0.0,0.0,9.0,0.0,53.88,2.1,0.500000,0.0,-0.793353,0.0,0.0,0.0,9.0,0.0,53.87,2.6,0.608761,0.0,-0.707107,0.0,0.0,-1.0,9.0,0.0,55.98,3.1,0.707107,0.0,0,0.0,0.0
52460,-0.500000,0.0,0.0,-8.0,9.0,0.0,82.26,1.2,0.866025,0.0,-0.382683,0.0,0.0,-8.0,9.0,0.0,82.25,1.3,0.923880,0.0,-0.258819,0.0,0.0,-8.0,9.0,0.0,81.14,1.5,0.965926,0.0,-0.130526,0.0,0.0,-8.0,9.0,0

In [70]:
X = dataset7.iloc[:,:-2]
X = encoding(X)
X.drop('season', axis =1, inplace=True)

y_1 = pd.DataFrame(dataset7.iloc[:,-2])
y_2 = pd.DataFrame(dataset7.iloc[:,-1])
X

,sin_time(t-144),TARGET(t-144),DNI(t-144),T(t-144),SH(t-144),DHI(t-144),RH(t-144),WS(t-144),cos_time(t-144),GHI(t-144),sin_time(t-143),TARGET(t-143),DNI(t-143),T(t-143),SH(t-143),DHI(t-143),RH(t-143),WS(t-143),cos_time(t-143),GHI(t-143),sin_time(t-142),TARGET(t-142),DNI(t-142),T(t-142),SH(t-142),DHI(t-142),RH(t-142),WS(t-142),cos_time(t-142),GHI(t-142),sin_time(t-141),TARGET(t-141),DNI(t-141),T(t-141),SH(t-141),DHI(t-141),RH(t-141),WS(t-141),cos_time(t-141),GHI(t-141),...,SH(t-2),DHI(t-2),RH(t-2),WS(t-2),cos_time(t-2),GHI(t-2),sin_time(t-1),TARGET(t-1),DNI(t-1),T(t-1),SH(t-1),DHI(t-1),RH(t-1),WS(t-1),cos_time(t-1),GHI(t-1),Season_0,Season_1,Season_2,Season_3,Season_4,Season_5,Season_6,Season_7,Season_8,Season_9,Season_10,Season_11,Season_12,Season_13,Season_14,Season_15,Season_16,Season_17,Season_18,Season_19,Season_20,Season_21,Season_22,Season_23
144,0.000000,0.0,0.0,-12.0,9.0,0.0,69.08,1.5,1.000000,0.0,0.130526,0.0,0.0,-12.0,9.0,0.0,69.06,1.5,0.991445,0.0,0.258819,0.0,0.0,-12.0,9.0,0.0,71.78,1.6,0.965926,0.0,0.382683,0.0,0.0,-12.0,9.0,0.0,71.75,1.6,0.923880,0.0,...,9.0,0.0,81.09,1.1,0.965926,0.0,-0.130526,0.0,0.0,-13.0,9.0,0.0,81.08,1.4,0.991445,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
145,0.130526,0.0,0.0,-12.0,9.0,0.0,69.06,1.5,0.991445,0.0,0.258819,0.0,0.0,-12.0,9.0,0.0,71.78,1.6,0.965926,0.0,0.382683,0.0,0.0,-12.0,9.0,0.0,71.75,1.6,0.923880,0.0,0.500000,0.0,0.0,-12.0,9.0,0.0,75.20,1.6,0.866025,0.0,...,9.0,0.0,81.08,1.4,0.991445,0.0,0.000000,0.0,0.0,-14.0,9.0,0.0,77.76,1.8,1.000000,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
146,0.258819,0.0,0.0,-12.0,9.0,0.0,71.78,1.6,0.965926,0.0,0.382683,0.0,0.0,-12.0,9.0,0.0,71.75,1.6,0.923880,0.0,0.500000,0.0,0.0,-12.0,9.0,0.0,75.20,1.6,0.866025,0.0,0.608761,0.0,0.0,-11.0,9.0,0.0,69.29,1.5,0.793353,0.0,...,9.0,0.0,77.76,1.8,1.000000,0.0,0.130526,0.0,0.0,-13.0,9.0,0.0,77.77,2.0,0.991445,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
147,0.382683,0.0,0.0,-12.0,9.0,0.0,71.75,1.6,0.923880,0.0,0.500000,0.0,0.0,-12.0,9.0,0.0,75.20,1.6,0.866025,0.0,0.608761,0.0,0.0,-11.0,9.0,0.0,69.29,1.5,0.793353,0.0,0.707107,0.0,0.0,-11.0,9.0,0.0,72.56,1.5,0.707107,0.0,...,9.0,0.0,77.77,2.0,0.991445,0.0,0.258819,0.0,0.0,-13.0,9.0,0.0,77.69,2.2,0.965926,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
148,0.500000,0.0,0.0,-12.0,9.0,0.0,75.20,1.6,0.866025,0.0,0.608761,0.0,0.0,-11.0,9.0,0.0,69.29,1.5,0.793353,0.0,0.707107,0.0,0.0,-11.0,9.0,0.0,72.56,1.5,0.707107,0.0,0.793353,0.0,0.0,-11.0,9.0,0.0,72.55,1.4,0.608761,0.0,...,9.0,0.0,77.69,2.2,0.965926,0.0,0.382683,0.0,0.0,-13.0,9.0,0.0,71.57,2.2,0.923880,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52459,-0.608761,0.0,0.0,-8.0,9.0,0.0,84.85,1.0,0.793353,0.0,-0.500000,0.0,0.0,-8.0,9.0,0.0,82.26,1.2,0.866025,0.0,-0.382683,0.0,0.0,-8.0,9.0,0.0,82.25,1.3,0.923880,0.0,-0.258819,0.0,0.0,-8.0,9.0,0.0,81.14,1.5,0.965926,0.0,...,9.0,0.0,53.87,2.6,0.608761,0.0,-0.707107,0.0,0.0,-1.0,9.0,0.0,55.98,3.1,0.707107,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
52460,-0.500000,0.0,0.0,-8.0,9.0,0.0,82.26,1.2,0.866025,0.0,-0.382683,0.0,0.0,-8.0,9.0,0.0,82.25,1.3,0.923880,0.0,-0.258819,0.0,0.0,-8.0,9.0,0.0,81.14,1.5,0.965926,0.0,-0.130526,0.0,0.0,-8.0,9.0,0.0,81.12,1.6,0.991445,0.0,...,9.0,0.0,55.98,3.1,0.707107,0.0,-0.608761,0.0,0.0,-1.0,9.0,0.0,55.97,3.5,0.793353,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
52461,-0.382683,0.0,0.0,-8.0,9.0,0.0,82.25,1.3,0.923880,0.0,-0.258819,0.0,0.0,-8.0,9.0,0.0,81.14,1.5,0.965926,0.0,-0.130526,0.0,0.0,-8.0,9.0,0.0,81.12,1.6,0.991445,0.0,0.000000,0.0,0.0,-8.0,9.0,0.0,80.36,1.8,1.000000,0.0,...,9.0,0.0,55.97,3.5,0.793353,0.0,-0.500000,0.0,0.0,-2.0,9.0,0.0,54.23,3.9,0.866025,0

# testset은 이미 24절기(season)가 들어있으므로, 약간 다르게 전처리를 해 줘야 함.
## (season) 구하는 과정이 생략됨.
## 이하는 testset을 구하는 코드

In [23]:
file_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/' + str(1) + '.csv'
new_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/adj_test/' + str(1) + '.csv'
file_name = str(1) + '.csv'

# adj_test에서 가져와야 하므로 new_path에서 불러온다
temp = pd.read_csv(new_path)
temp

,Unnamed: 0,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,Time,24
0,0,0,0,0,0,0,2.8,70.34,-6.1,0.0,0.0,22
1,1,0,0,30,0,0,2.9,71.97,-6.4,0.0,0.5,22
2,2,0,1,0,0,0,2.9,69.77,-6.7,0.0,1.0,22
3,3,0,1,30,0,0,2.9,71.95,-7.1,0.0,1.5,22
4,4,0,2,0,0,0,2.9,69.48,-7.4,0.0,2.0,22
...,...,...,...,...,...,...,...,...,...,...,...,...
331,331,6,21,30,0,0,1.7,43.65,-5.2,0.0,21.5,22
332,332,6,22,0,0,0,1.5,40.62,-4.9,0.0,22.0,22
333,333,6,22,30,0,0,1.5,40.62,-4.9,0.0,22.5,22
334,334,6,23,0,0,0,1.5,39.21,-4.9,0.0,23.0,22


In [24]:
temp.drop('Unnamed: 0', axis = 1, inplace = True)

In [25]:
testset = temp.copy()

# 24로 저장한 24절기 season으로 이름 바꾸기
testset.rename(columns = {'24' : 'season'}, inplace = True)


# 이 다음은 trainset과 동일한 전처리, season은 이미 구해져 있으므로 구하지 않음
testset2 = drop_clms(testset)
testset2.drop(['Hour', 'Minute','Date'], axis=1,inplace=True)
testset3 = cos_time(testset2)
testset4 = testset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]
testset4

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time
0,0,0,2.8,70.34,-6.1,0.0,9.5,0.000000,1.000000,22,0.0
1,0,0,2.9,71.97,-6.4,0.0,9.5,0.130526,0.991445,22,0.5
2,0,0,2.9,69.77,-6.7,0.0,9.5,0.258819,0.965926,22,1.0
3,0,0,2.9,71.95,-7.1,0.0,9.5,0.382683,0.923880,22,1.5
4,0,0,2.9,69.48,-7.4,0.0,9.5,0.500000,0.866025,22,2.0
...,...,...,...,...,...,...,...,...,...,...,...
331,0,0,1.7,43.65,-5.2,0.0,9.5,-0.608761,0.793353,22,21.5
332,0,0,1.5,40.62,-4.9,0.0,9.5,-0.500000,0.866025,22,22.0
333,0,0,1.5,40.62,-4.9,0.0,9.5,-0.382683,0.923880,22,22.5
334,0,0,1.5,39.21,-4.9,0.0,9.5,-0.258819,0.965926,22,23.0


In [26]:
testset5 = testset4.copy()
testset5['GHI'] = testset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)

In [27]:
testset5.iloc[50:51,:]

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time,GHI
50,0,0,2.1,61.33,-10.8,0.0,9.5,0.258819,0.965926,22,1.0,0.0


In [28]:
#### testset과 dataset이 같은 모양인지 중간점검
dataset5.iloc[50:51,:]

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time,GHI
50,0,0,1.6,88.11,-11,0.0,9.0,0.258819,0.965926,23,1.0,0.0


In [29]:
testset6 = testset5[['DHI', 'DNI', 'WS', 'RH', 'T','SH', 'sin_time', 'cos_time', 'GHI', 'season', 'TARGET']]
testset6.head(10)

,DHI,DNI,WS,RH,T,SH,sin_time,cos_time,GHI,season,TARGET
0,0,0,2.8,70.34,-6.1,9.5,0.000000,1.000000,0.0,22,0.0
1,0,0,2.9,71.97,-6.4,9.5,0.130526,0.991445,0.0,22,0.0
2,0,0,2.9,69.77,-6.7,9.5,0.258819,0.965926,0.0,22,0.0
3,0,0,2.9,71.95,-7.1,9.5,0.382683,0.923880,0.0,22,0.0
4,0,0,2.9,69.48,-7.4,9.5,0.500000,0.866025,0.0,22,0.0
5,0,0,2.9,72.13,-7.9,9.5,0.608761,0.793353,0.0,22,0.0
6,0,0,2.9,69.76,-8.3,9.5,0.707107,0.707107,0.0,22,0.0
7,0,0,2.9,71.97,-8.7,9.5,0.793353,0.608761,0.0,22,0.0
8,0,0,2.9,68.56,-9.1,9.5,0.866025,0.500000,0.0,22,0.0
9,0,0,2.7,70.75,-9.5,9.5,0.923880,0.382683,0.0,22,0.0


In [30]:
testset7 = test_to_supervised(testset6, n_days)
testset7

,sin_time(t-144),TARGET(t-144),DNI(t-144),T(t-144),SH(t-144),DHI(t-144),RH(t-144),WS(t-144),cos_time(t-144),GHI(t-144),sin_time(t-143),TARGET(t-143),DNI(t-143),T(t-143),SH(t-143),DHI(t-143),RH(t-143),WS(t-143),cos_time(t-143),GHI(t-143),sin_time(t-142),TARGET(t-142),DNI(t-142),T(t-142),SH(t-142),DHI(t-142),RH(t-142),WS(t-142),cos_time(t-142),GHI(t-142),sin_time(t-141),TARGET(t-141),DNI(t-141),T(t-141),SH(t-141),DHI(t-141),RH(t-141),WS(t-141),cos_time(t-141),GHI(t-141),...,TARGET(t-4),DNI(t-4),T(t-4),SH(t-4),DHI(t-4),RH(t-4),WS(t-4),cos_time(t-4),GHI(t-4),sin_time(t-3),TARGET(t-3),DNI(t-3),T(t-3),SH(t-3),DHI(t-3),RH(t-3),WS(t-3),cos_time(t-3),GHI(t-3),sin_time(t-2),TARGET(t-2),DNI(t-2),T(t-2),SH(t-2),DHI(t-2),RH(t-2),WS(t-2),cos_time(t-2),GHI(t-2),sin_time(t-1),TARGET(t-1),DNI(t-1),T(t-1),SH(t-1),DHI(t-1),RH(t-1),WS(t-1),cos_time(t-1),GHI(t-1),season
144,0.000000,0.0,0.0,-6.1,9.5,0.0,70.34,2.8,1.000000,0.0,0.130526,0.0,0.0,-6.4,9.5,0.0,71.97,2.9,0.991445,0.0,0.258819,0.0,0.0,-6.7,9.5,0.0,69.77,2.9,0.965926,0.0,0.382683,0.0,0.0,-7.1,9.5,0.0,71.95,2.9,0.923880,0.0,...,0.0,0.0,-4.5,9.5,0.0,52.07,4.0,0.866025,0.0,-0.382683,0.0,0.0,-4.5,9.5,0.0,52.01,3.9,0.923880,0.0,-0.258819,0.0,0.0,-4.4,9.5,0.0,53.72,3.9,0.965926,0.0,-0.130526,0.0,0.0,-4.3,9.5,0.0,53.31,3.9,0.991445,0.0,22
145,0.130526,0.0,0.0,-6.4,9.5,0.0,71.97,2.9,0.991445,0.0,0.258819,0.0,0.0,-6.7,9.5,0.0,69.77,2.9,0.965926,0.0,0.382683,0.0,0.0,-7.1,9.5,0.0,71.95,2.9,0.923880,0.0,0.500000,0.0,0.0,-7.4,9.5,0.0,69.48,2.9,0.866025,0.0,...,0.0,0.0,-4.5,9.5,0.0,52.01,3.9,0.923880,0.0,-0.258819,0.0,0.0,-4.4,9.5,0.0,53.72,3.9,0.965926,0.0,-0.130526,0.0,0.0,-4.3,9.5,0.0,53.31,3.9,0.991445,0.0,0.000000,0.0,0.0,-4.2,9.5,0.0,54.93,3.9,1.000000,0.0,22
146,0.258819,0.0,0.0,-6.7,9.5,0.0,69.77,2.9,0.965926,0.0,0.382683,0.0,0.0,-7.1,9.5,0.0,71.95,2.9,0.923880,0.0,0.500000,0.0,0.0,-7.4,9.5,0.0,69.48,2.9,0.866025,0.0,0.608761,0.0,0.0,-7.9,9.5,0.0,72.13,2.9,0.793353,0.0,...,0.0,0.0,-4.4,9.5,0.0,53.72,3.9,0.965926,0.0,-0.130526,0.0,0.0,-4.3,9.5,0.0,53.31,3.9,0.991445,0.0,0.000000,0.0,0.0,-4.2,9.5,0.0,54.93,3.9,1.000000,0.0,0.130526,0.0,0.0,-4.1,9.5,0.0,54.52,3.8,0.991445,0.0,22
147,0.382683,0.0,0.0,-7.1,9.5,0.0,71.95,2.9,0.923880,0.0,0.500000,0.0,0.0,-7.4,9.5,0.0,69.48,2.9,0.866025,0.0,0.608761,0.0,0.0,-7.9,9.5,0.0,72.13,2.9,0.793353,0.0,0.707107,0.0,0.0,-8.3,9.5,0.0,69.76,2.9,0.707107,0.0,...,0.0,0.0,-4.3,9.5,0.0,53.31,3.9,0.991445,0.0,0.000000,0.0,0.0,-4.2,9.5,0.0,54.93,3.9,1.000000,0.0,0.130526,0.0,0.0,-4.1,9.5,0.0,54.52,3.8,0.991445,0.0,0.258819,0.0,0.0,-4.0,9.5,0.0,56.81,3.7,0.965926,0.0,22
148,0.500000,0.0,0.0,-7.4,9.5,0.0,69.48,2.9,0.866025,0.0,0.608761,0.0,0.0,-7.9,9.5,0.0,72.13,2.9,0.793353,0.0,0.707107,0.0,0.0,-8.3,9.5,0.0,69.76,2.9,0.707107,0.0,0.793353,0.0,0.0,-8.7,9.5,0.0,71.97,2.9,0.608761,0.0,...,0.0,0.0,-4.2,9.5,0.0,54.93,3.9,1.000000,0.0,0.130526,0.0,0.0,-4.1,9.5,0.0,54.52,3.8,0.991445,0.0,0.258819,0.0,0.0,-4.0,9.5,0.0,56.81,3.7,0.965926,0.0,0.382683,0.0,0.0,-3.8,9.5,0.0,55.90,3.6,0.923880,0.0,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,-0.608761,0.0,0.0,-3.5,9.5,0.0,60.79,0.6,0.793353,0.0,-0.500000,0.0,0.0,-3.8,9.5,0.0,60.30,0.6,0.866025,0.0,-0.382683,0.0,0.0,-4.1,9.5,0.0,61.60,0.6,0.923880,0.0,-0.258819,0.0,0.0,-4.4,9.5,0.0,61.43,0.6,0.965926,0.0,...,0.0,0.0,-5.2,9.5,0.0,42.88,1.5,0.382683,0.0,-0.866025,0.0,0.0,-5.4,9.5,0.0,45.58,1.6,0.500000,0.0,-0.793353,0.0,0.0,-5.4,9.5,0.0,45.58,1.7,0.608761,0.0,-0.707107,0.0,0.0,-5.4,9.5,0.0,44.37,1.8,0.707107,0.0,22
332,-0.500000,0.0,0.0,-3.8,9.5,0.0,60.30,0.6,0.866025,0.0,-0.382683,0.0,0.0,-4.1,9.5,0.0,61.60,0.6,0.923880,0.0,-0.258819,0.0,0.0,-4.4,9.5,0.0,61.43,0.6,0.965926,0.0,-0.130526,0.0,0.0,-4.7,9.5,0.0,62.83,0.6,0.991445,0.0,...,0.0,0.0,-5.4,9.5,

In [31]:
testset8 = encoding(testset7)
testset8.drop('season', axis = 1, inplace = True)
testset8

,sin_time(t-144),TARGET(t-144),DNI(t-144),T(t-144),SH(t-144),DHI(t-144),RH(t-144),WS(t-144),cos_time(t-144),GHI(t-144),sin_time(t-143),TARGET(t-143),DNI(t-143),T(t-143),SH(t-143),DHI(t-143),RH(t-143),WS(t-143),cos_time(t-143),GHI(t-143),sin_time(t-142),TARGET(t-142),DNI(t-142),T(t-142),SH(t-142),DHI(t-142),RH(t-142),WS(t-142),cos_time(t-142),GHI(t-142),sin_time(t-141),TARGET(t-141),DNI(t-141),T(t-141),SH(t-141),DHI(t-141),RH(t-141),WS(t-141),cos_time(t-141),GHI(t-141),...,SH(t-2),DHI(t-2),RH(t-2),WS(t-2),cos_time(t-2),GHI(t-2),sin_time(t-1),TARGET(t-1),DNI(t-1),T(t-1),SH(t-1),DHI(t-1),RH(t-1),WS(t-1),cos_time(t-1),GHI(t-1),Season_0,Season_1,Season_2,Season_3,Season_4,Season_5,Season_6,Season_7,Season_8,Season_9,Season_10,Season_11,Season_12,Season_13,Season_14,Season_15,Season_16,Season_17,Season_18,Season_19,Season_20,Season_21,Season_22,Season_23
144,0.000000,0.0,0.0,-6.1,9.5,0.0,70.34,2.8,1.000000,0.0,0.130526,0.0,0.0,-6.4,9.5,0.0,71.97,2.9,0.991445,0.0,0.258819,0.0,0.0,-6.7,9.5,0.0,69.77,2.9,0.965926,0.0,0.382683,0.0,0.0,-7.1,9.5,0.0,71.95,2.9,0.923880,0.0,...,9.5,0.0,53.72,3.9,0.965926,0.0,-0.130526,0.0,0.0,-4.3,9.5,0.0,53.31,3.9,0.991445,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
145,0.130526,0.0,0.0,-6.4,9.5,0.0,71.97,2.9,0.991445,0.0,0.258819,0.0,0.0,-6.7,9.5,0.0,69.77,2.9,0.965926,0.0,0.382683,0.0,0.0,-7.1,9.5,0.0,71.95,2.9,0.923880,0.0,0.500000,0.0,0.0,-7.4,9.5,0.0,69.48,2.9,0.866025,0.0,...,9.5,0.0,53.31,3.9,0.991445,0.0,0.000000,0.0,0.0,-4.2,9.5,0.0,54.93,3.9,1.000000,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
146,0.258819,0.0,0.0,-6.7,9.5,0.0,69.77,2.9,0.965926,0.0,0.382683,0.0,0.0,-7.1,9.5,0.0,71.95,2.9,0.923880,0.0,0.500000,0.0,0.0,-7.4,9.5,0.0,69.48,2.9,0.866025,0.0,0.608761,0.0,0.0,-7.9,9.5,0.0,72.13,2.9,0.793353,0.0,...,9.5,0.0,54.93,3.9,1.000000,0.0,0.130526,0.0,0.0,-4.1,9.5,0.0,54.52,3.8,0.991445,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
147,0.382683,0.0,0.0,-7.1,9.5,0.0,71.95,2.9,0.923880,0.0,0.500000,0.0,0.0,-7.4,9.5,0.0,69.48,2.9,0.866025,0.0,0.608761,0.0,0.0,-7.9,9.5,0.0,72.13,2.9,0.793353,0.0,0.707107,0.0,0.0,-8.3,9.5,0.0,69.76,2.9,0.707107,0.0,...,9.5,0.0,54.52,3.8,0.991445,0.0,0.258819,0.0,0.0,-4.0,9.5,0.0,56.81,3.7,0.965926,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
148,0.500000,0.0,0.0,-7.4,9.5,0.0,69.48,2.9,0.866025,0.0,0.608761,0.0,0.0,-7.9,9.5,0.0,72.13,2.9,0.793353,0.0,0.707107,0.0,0.0,-8.3,9.5,0.0,69.76,2.9,0.707107,0.0,0.793353,0.0,0.0,-8.7,9.5,0.0,71.97,2.9,0.608761,0.0,...,9.5,0.0,56.81,3.7,0.965926,0.0,0.382683,0.0,0.0,-3.8,9.5,0.0,55.90,3.6,0.923880,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,-0.608761,0.0,0.0,-3.5,9.5,0.0,60.79,0.6,0.793353,0.0,-0.500000,0.0,0.0,-3.8,9.5,0.0,60.30,0.6,0.866025,0.0,-0.382683,0.0,0.0,-4.1,9.5,0.0,61.60,0.6,0.923880,0.0,-0.258819,0.0,0.0,-4.4,9.5,0.0,61.43,0.6,0.965926,0.0,...,9.5,0.0,45.58,1.7,0.608761,0.0,-0.707107,0.0,0.0,-5.4,9.5,0.0,44.37,1.8,0.707107,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
332,-0.500000,0.0,0.0,-3.8,9.5,0.0,60.30,0.6,0.866025,0.0,-0.382683,0.0,0.0,-4.1,9.5,0.0,61.60,0.6,0.923880,0.0,-0.258819,0.0,0.0,-4.4,9.5,0.0,61.43,0.6,0.965926,0.0,-0.130526,0.0,0.0,-4.7,9.5,0.0,62.83,0.6,0.991445,0.0,...,9.5,0.0,44.37,1.8,0.707107,0.0,-0.608761,0.0,0.0,-5.2,9.5,0.0,43.65,1.7,0.793353,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
333,-0.382683,0.0,0.0,-4.1,9.5,0.0,61.60,0.6,0.923880,0.0,-0.258819,0.0,0.0,-4.4,9.5,0.0,61.43,0.6,0.965926,0.0,-0.130526,0.0,0.0,-4.7,9.5,0.0,62.83,0.6,0.991445,0.0,0.000000,0.0,0.0,-4.9,9.5,0.0,62.81,0.7,1.000000,0.0,...,9.5,0.0,43.65,1.7,0.793353,0.0,-0.500000,0.0,0.0,-4.9,9.5,0.0,40.62,1.5,0.866025,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [32]:
testset9 = testset8.iloc[-48:,:]
testset9

,sin_time(t-144),TARGET(t-144),DNI(t-144),T(t-144),SH(t-144),DHI(t-144),RH(t-144),WS(t-144),cos_time(t-144),GHI(t-144),sin_time(t-143),TARGET(t-143),DNI(t-143),T(t-143),SH(t-143),DHI(t-143),RH(t-143),WS(t-143),cos_time(t-143),GHI(t-143),sin_time(t-142),TARGET(t-142),DNI(t-142),T(t-142),SH(t-142),DHI(t-142),RH(t-142),WS(t-142),cos_time(t-142),GHI(t-142),sin_time(t-141),TARGET(t-141),DNI(t-141),T(t-141),SH(t-141),DHI(t-141),RH(t-141),WS(t-141),cos_time(t-141),GHI(t-141),...,SH(t-2),DHI(t-2),RH(t-2),WS(t-2),cos_time(t-2),GHI(t-2),sin_time(t-1),TARGET(t-1),DNI(t-1),T(t-1),SH(t-1),DHI(t-1),RH(t-1),WS(t-1),cos_time(t-1),GHI(t-1),Season_0,Season_1,Season_2,Season_3,Season_4,Season_5,Season_6,Season_7,Season_8,Season_9,Season_10,Season_11,Season_12,Season_13,Season_14,Season_15,Season_16,Season_17,Season_18,Season_19,Season_20,Season_21,Season_22,Season_23
288,0.000000e+00,0.000000,0.0,-4.2,9.5,0.0,54.93,3.9,1.000000e+00,0.000000,1.305262e-01,0.000000,0.0,-4.1,9.5,0.0,54.52,3.8,9.914449e-01,0.000000,2.588190e-01,0.000000,0.0,-4.0,9.5,0.0,56.81,3.7,9.659258e-01,0.000000,3.826834e-01,0.000000,0.0,-3.8,9.5,0.0,55.90,3.6,9.238795e-01,0.000000,...,9.5,0.0,55.81,1.8,9.659258e-01,0.000000,-1.305262e-01,0.000000,0.0,-11.0,9.5,0.0,54.92,1.7,9.914449e-01,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
289,1.305262e-01,0.000000,0.0,-4.1,9.5,0.0,54.52,3.8,9.914449e-01,0.000000,2.588190e-01,0.000000,0.0,-4.0,9.5,0.0,56.81,3.7,9.659258e-01,0.000000,3.826834e-01,0.000000,0.0,-3.8,9.5,0.0,55.90,3.6,9.238795e-01,0.000000,5.000000e-01,0.000000,0.0,-3.7,9.5,0.0,58.48,3.4,8.660254e-01,0.000000,...,9.5,0.0,54.92,1.7,9.914449e-01,0.000000,0.000000e+00,0.000000,0.0,-10.7,9.5,0.0,52.40,1.7,1.000000e+00,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
290,2.588190e-01,0.000000,0.0,-4.0,9.5,0.0,56.81,3.7,9.659258e-01,0.000000,3.826834e-01,0.000000,0.0,-3.8,9.5,0.0,55.90,3.6,9.238795e-01,0.000000,5.000000e-01,0.000000,0.0,-3.7,9.5,0.0,58.48,3.4,8.660254e-01,0.000000,6.087614e-01,0.000000,0.0,-3.6,9.5,0.0,58.05,3.2,7.933533e-01,0.000000,...,9.5,0.0,52.40,1.7,1.000000e+00,0.000000,1.305262e-01,0.000000,0.0,-10.5,9.5,0.0,51.58,1.7,9.914449e-01,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
291,3.826834e-01,0.000000,0.0,-3.8,9.5,0.0,55.90,3.6,9.238795e-01,0.000000,5.000000e-01,0.000000,0.0,-3.7,9.5,0.0,58.48,3.4,8.660254e-01,0.000000,6.087614e-01,0.000000,0.0,-3.6,9.5,0.0,58.05,3.2,7.933533e-01,0.000000,7.071068e-01,0.000000,0.0,-3.6,9.5,0.0,60.69,3.0,7.071068e-01,0.000000,...,9.5,0.0,51.58,1.7,9.914449e-01,0.000000,2.588190e-01,0.000000,0.0,-10.3,9.5,0.0,49.37,1.7,9.659258e-01,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
292,5.000000e-01,0.000000,0.0,-3.7,9.5,0.0,58.48,3.4,8.660254e-01,0.000000,6.087614e-01,0.000000,0.0,-3.6,9.5,0.0,58.05,3.2,7.933533e-01,0.000000,7.071068e-01,0.000000,0.0,-3.6,9.5,0.0,60.69,3.0,7.071068e-01,0.000000,7.933533e-01,0.000000,0.0,-3.5,9.5,0.0,60.24,3.0,6.087614e-01,0.000000,...,9.5,0.0,49.37,1.7,9.659258e-01,0.000000,3.826834e-01,0.000000,0.0,-10.1,9.5,0.0,48.59,1.7,9.238795e-01,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
293,6.087614e-01,0.000000,0.0,-3.6,9.5,0.0,58.05,3.2,7.933533e-01,0.000000,7.071068e-01,0.000000,0.0,-3.6,9.5,0.0,60.69,3.0,7.071068e-01,0.000000,7.933533e-01,0.000000,0.0,-3.5,9.5,0.0,60.24,3.0,6.087614e-01,0.000000,8.660254e-01,0.000000,0.0,-3.5,9.5,0.0,61.65,2.9,5.000000e-01,0.000000,...,9.5,0.0,48.59,1.7,9.238795e-01,0.000000,5.000000e-01,0.000000,0.0,-10.0,9.5,0.0,47.31,1.7,8.660254e-01,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
294,7.071068e-01,0.000000,0.0,-3.6,9.5,0.0,60.69,3.0,7.071068e-01,0.000000,7.933533e-01,0.000000,0.0,-3.5,9.5,0.0,60.24,3.0,6.087614e-01,0.000000,8.660254e-01,0.000000,0.0,-3.5,9.5,0.0,61.65,2.9,5.000000e-01,0.000000,9.238795e-01,0.000000,0.0,-3.4,9.5,0.0,61.19,3.0,3.826834e-01,0.000000,...,9.5,0.0,47.31,1.7,8.660254e-01,0.000000,6.087614e-01,0.000000,0.0,-9.8,9.5,0.0,46.57,1.6,7.933533e-01,0.000000,0,0,0,0,0,0,0,0,0,0,0,0

In [80]:
df_test = []
df_valid_X = []
df_valid_y1 = []
df_valid_y2 = []
# 예시 코드
# 아직 유진이 어떻게 변수 추가할 지 몰라서 to_supervised는 쓰지 않았음.


for i in range(81):
    file_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/' + str(i) + '.csv'
    new_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/adj_test/' + str(i) + '.csv'
    
    # adj_test에서 가져와야 하므로 new_path에서 불러온다
    temp = pd.read_csv(new_path)
    temp.drop('Unnamed: 0', axis = 1, inplace = True)
    testset = temp.copy()

    # 24로 저장한 24절기 season으로 이름 바꾸기
    testset.rename(columns = {'24' : 'season'}, inplace = True)

    # 이 다음은 trainset과 동일한 전처리, season은 이미 구해져 있으므로 구하지 않음
    testset2 = drop_clms(testset)
    testset2.drop(['Hour', 'Minute','Date'], axis=1,inplace=True)
    testset3 = cos_time(testset2)
    testset4 = testset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]

    testset5 = testset4.copy()
    testset5['GHI'] = testset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)
    testset6 = testset5[['DHI', 'DNI', 'WS', 'RH', 'T','SH', 'sin_time', 'cos_time', 'GHI', 'season', 'TARGET']]
    testset7 = test_to_supervised(testset6, n_days)
    
    testset8 = encoding(testset7)
    testset8.drop('season', axis = 1, inplace = True)
    testset9 = testset8.iloc[-48:,:]

    #############################
    validset = train_to_supervised(testset6, n_days)
    X_valid = validset.iloc[:,:-2]
    X_valid = encoding(X_valid)
    X_valid.drop('season', axis =1, inplace=True)

    valid_y_1 = pd.DataFrame(validset.iloc[:,-2])
    valid_y_2 = pd.DataFrame(validset.iloc[:,-1])

    df_valid_X.append(X_valid)
    df_valid_y1.append(valid_y_1)
    df_valid_y2.append(valid_y_2)
    df_test.append(testset9)
    ############################

X_test = pd.concat(df_test)
X_valid_1t = pd.concat(df_valid_X)
X_valid_2t = X_valid_1t.copy()
Y_valid_1t = pd.concat(df_valid_y1)
Y_valid_2t = pd.concat(df_valid_y2)
# X_test = X_test.iloc[:, :n_obs]
X_test 

,sin_time(t-144),TARGET(t-144),DNI(t-144),T(t-144),SH(t-144),DHI(t-144),RH(t-144),WS(t-144),cos_time(t-144),GHI(t-144),sin_time(t-143),TARGET(t-143),DNI(t-143),T(t-143),SH(t-143),DHI(t-143),RH(t-143),WS(t-143),cos_time(t-143),GHI(t-143),sin_time(t-142),TARGET(t-142),DNI(t-142),T(t-142),SH(t-142),DHI(t-142),RH(t-142),WS(t-142),cos_time(t-142),GHI(t-142),sin_time(t-141),TARGET(t-141),DNI(t-141),T(t-141),SH(t-141),DHI(t-141),RH(t-141),WS(t-141),cos_time(t-141),GHI(t-141),...,SH(t-2),DHI(t-2),RH(t-2),WS(t-2),cos_time(t-2),GHI(t-2),sin_time(t-1),TARGET(t-1),DNI(t-1),T(t-1),SH(t-1),DHI(t-1),RH(t-1),WS(t-1),cos_time(t-1),GHI(t-1),Season_0,Season_1,Season_2,Season_3,Season_4,Season_5,Season_6,Season_7,Season_8,Season_9,Season_10,Season_11,Season_12,Season_13,Season_14,Season_15,Season_16,Season_17,Season_18,Season_19,Season_20,Season_21,Season_22,Season_23
288,0.000000,0.0,0.0,0.1,9.5,0.0,59.09,1.1,1.000000,0.0,0.130526,0.0,0.0,-0.4,9.5,0.0,61.20,1.2,0.991445,0.0,0.258819,0.0,0.0,-0.8,9.5,0.0,65.76,1.3,0.965926,0.0,0.382683,0.0,0.0,-0.9,9.5,0.0,66.24,1.4,0.923880,0.0,...,9.5,0.0,81.38,0.7,0.965926,0.0,-0.130526,0.0,0.0,-2.7,9.5,0.0,81.99,0.8,0.991445,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
289,0.130526,0.0,0.0,-0.4,9.5,0.0,61.20,1.2,0.991445,0.0,0.258819,0.0,0.0,-0.8,9.5,0.0,65.76,1.3,0.965926,0.0,0.382683,0.0,0.0,-0.9,9.5,0.0,66.24,1.4,0.923880,0.0,0.500000,0.0,0.0,-1.0,9.5,0.0,69.70,1.5,0.866025,0.0,...,9.5,0.0,81.99,0.8,0.991445,0.0,0.000000,0.0,0.0,-2.8,9.5,0.0,80.92,0.8,1.000000,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
290,0.258819,0.0,0.0,-0.8,9.5,0.0,65.76,1.3,0.965926,0.0,0.382683,0.0,0.0,-0.9,9.5,0.0,66.24,1.4,0.923880,0.0,0.500000,0.0,0.0,-1.0,9.5,0.0,69.70,1.5,0.866025,0.0,0.608761,0.0,0.0,-1.0,9.5,0.0,69.70,1.6,0.793353,0.0,...,9.5,0.0,80.92,0.8,1.000000,0.0,0.130526,0.0,0.0,-2.9,9.5,0.0,81.53,0.9,0.991445,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
291,0.382683,0.0,0.0,-0.9,9.5,0.0,66.24,1.4,0.923880,0.0,0.500000,0.0,0.0,-1.0,9.5,0.0,69.70,1.5,0.866025,0.0,0.608761,0.0,0.0,-1.0,9.5,0.0,69.70,1.6,0.793353,0.0,0.707107,0.0,0.0,-1.0,9.5,0.0,68.17,1.7,0.707107,0.0,...,9.5,0.0,81.53,0.9,0.991445,0.0,0.258819,0.0,0.0,-3.0,9.5,0.0,79.91,1.0,0.965926,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
292,0.500000,0.0,0.0,-1.0,9.5,0.0,69.70,1.5,0.866025,0.0,0.608761,0.0,0.0,-1.0,9.5,0.0,69.70,1.6,0.793353,0.0,0.707107,0.0,0.0,-1.0,9.5,0.0,68.17,1.7,0.707107,0.0,0.793353,0.0,0.0,-0.9,9.5,0.0,67.68,1.8,0.608761,0.0,...,9.5,0.0,79.91,1.0,0.965926,0.0,0.382683,0.0,0.0,-3.0,9.5,0.0,79.91,0.9,0.923880,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,-0.608761,0.0,0.0,12.2,15.0,0.0,79.32,1.1,0.793353,0.0,-0.500000,0.0,0.0,11.7,15.0,0.0,80.41,1.2,0.866025,0.0,-0.382683,0.0,0.0,11.2,15.0,0.0,83.10,1.1,0.923880,0.0,-0.258819,0.0,0.0,10.8,15.0,0.0,84.37,1.1,0.965926,0.0,...,15.5,0.0,59.77,0.8,0.608761,0.0,-0.707107,0.0,0.0,14.2,15.5,0.0,61.34,0.8,0.707107,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
332,-0.500000,0.0,0.0,11.7,15.0,0.0,80.41,1.2,0.866025,0.0,-0.382683,0.0,0.0,11.2,15.0,0.0,83.10,1.1,0.923880,0.0,-0.258819,0.0,0.0,10.8,15.0,0.0,84.37,1.1,0.965926,0.0,-0.130526,0.0,0.0,10.6,15.0,0.0,85.50,1.1,0.991445,0.0,...,15.5,0.0,61.34,0.8,0.707107,0.0,-0.608761,0.0,0.0,13.7,15.5,0.0,63.35,0.8,0.793353,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
333,-0.382683,0.0,0.0,11.2,15.0,0.0,83.10,1.1,0.923880,0.0,-0.258819,0.0,0.0,10.8,15.0,0.0,84.37,1.1,0.965926,0.0,-0.130526,0.0,0.0,10.6,15.0,0.0,85.50,1.1,0.991445,0.0,0.000000,0.0,0.0,10.3,15.0,0.0,86.41,1.1,1.000000,0.0,...,15.5,0.0,63.35,0.8,0.793353,0.0,-0.500000,0.0,0.0,13.1,15.5,0.0,64.82,0.7,0.866025,0.0,0,0,0,0,0,0

In [79]:
X_valid_2

,sin_time(t-144),TARGET(t-144),DNI(t-144),T(t-144),SH(t-144),DHI(t-144),RH(t-144),WS(t-144),cos_time(t-144),GHI(t-144),sin_time(t-143),TARGET(t-143),DNI(t-143),T(t-143),SH(t-143),DHI(t-143),RH(t-143),WS(t-143),cos_time(t-143),GHI(t-143),sin_time(t-142),TARGET(t-142),DNI(t-142),T(t-142),SH(t-142),DHI(t-142),RH(t-142),WS(t-142),cos_time(t-142),GHI(t-142),sin_time(t-141),TARGET(t-141),DNI(t-141),T(t-141),SH(t-141),DHI(t-141),RH(t-141),WS(t-141),cos_time(t-141),GHI(t-141),...,SH(t-2),DHI(t-2),RH(t-2),WS(t-2),cos_time(t-2),GHI(t-2),sin_time(t-1),TARGET(t-1),DNI(t-1),T(t-1),SH(t-1),DHI(t-1),RH(t-1),WS(t-1),cos_time(t-1),GHI(t-1),Season_0,Season_1,Season_2,Season_3,Season_4,Season_5,Season_6,Season_7,Season_8,Season_9,Season_10,Season_11,Season_12,Season_13,Season_14,Season_15,Season_16,Season_17,Season_18,Season_19,Season_20,Season_21,Season_22,Season_23
29851,-0.608761,0.000000,0.0,15.0,12.5,0.0,45.25,2.5,0.793353,0.000000,-5.000000e-01,0.000000,0.0,15.0,12.5,0.0,44.48,2.8,0.866025,0.000000,-0.382683,0.000000,0.0,14.0,12.5,0.0,44.48,2.8,0.923880,0.000000,-0.258819,0.000000,0.0,14.0,12.5,0.0,45.53,2.9,0.965926,0.000000,...,12.5,0.0,66.47,1.6,0.608761,0.000000,-0.707107,0.000000,0.0,12.0,12.5,0.0,66.51,1.7,0.707107,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
9985,0.130526,0.000000,0.0,14.0,14.0,0.0,71.73,3.2,0.991445,0.000000,2.588190e-01,0.000000,0.0,14.0,14.0,0.0,71.54,3.3,0.965926,0.000000,0.382683,0.000000,0.0,14.0,14.0,0.0,76.36,3.4,0.923880,0.000000,0.500000,0.000000,0.0,14.0,14.0,0.0,75.83,3.5,0.866025,0.000000,...,14.0,0.0,53.45,2.2,0.991445,0.000000,0.000000,0.000000,0.0,15.0,14.0,0.0,51.85,2.2,1.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
29105,0.793353,52.917987,841.0,21.0,12.5,89.0,33.14,1.9,-0.608761,684.723039,7.071068e-01,61.174022,871.0,22.0,12.5,95.0,30.21,1.8,-0.707107,750.141508,0.608761,68.491634,894.0,23.0,12.5,99.0,28.40,1.9,-0.793353,806.961399,0.500000,60.046800,406.0,24.0,12.5,331.0,26.19,2.0,-0.866025,666.707369,...,12.5,62.0,66.98,1.1,-0.382683,530.406995,0.866025,42.972881,824.0,20.0,12.5,67.0,53.29,1.3,-0.500000,611.820906,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
43751,0.130526,10.132340,0.0,29.0,14.5,107.0,18.85,3.7,-0.991445,107.000000,1.224647e-16,84.435243,392.0,30.0,14.5,523.0,18.40,3.6,-1.000000,823.422388,-0.130526,73.364844,323.0,30.0,14.5,474.0,18.40,3.3,-0.991445,722.153177,-0.258819,24.392403,17.0,30.0,14.5,244.0,18.14,3.0,-0.965926,257.036082,...,14.5,92.0,15.13,1.9,-0.923880,820.668979,0.258819,95.132465,991.0,28.0,14.5,92.0,13.56,2.1,-0.965926,837.989493,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4351,-0.793353,52.076413,931.0,16.0,12.5,70.0,20.57,1.0,-0.608761,840.713257,-8.660254e-01,42.505613,882.0,16.0,12.5,66.0,21.04,0.7,-0.500000,754.868513,-0.923880,32.466370,820.0,14.0,12.5,59.0,23.97,0.6,-0.382683,656.681488,-0.965926,22.144935,723.0,13.0,12.5,51.0,35.83,0.6,-0.258819,537.055191,...,12.5,320.0,30.14,3.9,-0.793353,597.909548,-0.707107,44.195064,317.0,15.0,12.5,278.0,30.94,4.0,-0.707107,553.311402,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,-0.608761,0.000000,0.0,9.3,15.0,0.0,68.24,1.2,0.793353,0.000000,-5.000000e-01,0.000000,0.0,8.7,15.0,0.0,69.78,1.2,0.866025,0.000000,-0.382683,0.000000,0.0,8.3,15.0,0.0,71.70,1.3,0.923880,0.000000,-0.258819,0.000000,0.0,7.9,15.0,0.0,71.52,1.4,0.965926,0.000000,...,15.0,0.0,57.06,0.9,0.608761,0.000000,-0.707107,0.000000,0.0,13.8,15.0,0.0,58.70,1.0,0.707107,0.000000,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
236,-0.500000,0.000000,0.0,8.7,15.0,0.0,69.78,1.2,0.866025,0.000000,-3.826834e-01,0.000000,0.0,8.3,15.0,0.0,71.70,1.3,0.923880,0.000000,-0.258819,0.000000,0.0,7.9,15.0

In [78]:
Y_valid_2

,TARGET1(t+96)
29851,0.000000
9985,0.000000
29105,51.227989
43751,76.836915
4351,55.272147
...,...
235,0.000000
236,0.000000
237,0.000000
238,0.000000


In [81]:
X_train_1, X_valid_1, Y_train_1,  Y_valid_1 = train_test_split(X, y_1, test_size=0.20, random_state=42)
X_train_2, X_valid_2, Y_train_2,  Y_valid_2 = train_test_split(X, y_2, test_size=0.20, random_state=42)


In [83]:


X_valid_1 = pd.concat([X_valid_1, X_valid_1t], axis = 0)
Y_valid_1 = pd.concat([Y_valid_1, Y_valid_1t], axis = 0)
X_valid_2 = pd.concat([X_valid_2, X_valid_2t], axis = 0)
Y_valid_2 = pd.concat([Y_valid_2, Y_valid_2t], axis = 0)



quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [65]:
# LGBM 모델을 구축하고 예측까지 하는 함수를 만든다. 
def LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test):
    
    # (a) 모델링
    model = LGBMRegressor(objective='quantile', alpha=q,
                         n_estimators=10000, bagging_fraction=0.6, learning_rate=0.02, save_binary = True, subsample=0.7, device = 'gpu')          
    
        # bagging_fraction : 배깅을 하기위해서 데이터를 랜덤 샘플링하여 학습에 사용한다. 비율은 0 < fraction <= 1 이며 0이 되지 않게 해야한다.
        # learning_rate : 일반적으로 0.01 ~ 0.1 정도로 맞추고 다른 파라미터를 튜닝한다. 나중에 성능을 더 높일 때 learning rate를 더 줄인다.
        # n_estimators : 기본값은 100, 1000 정도로 해주는 게 좋고 많을 수록 과적합이 발생한 가능성이 높음.
        # save_binary = True 넣어주면 더 빨라진다고 함.
        # https://greatjoy.tistory.com/72
        
    model.fit(X_train, Y_train, eval_metric = ['quantile'], eval_set=[(X_valid, Y_valid)], early_stopping_rounds=500, verbose=500)


        # verbose : eval metric이 이 숫자만큼의 round가 지난 다음 자동으로 출력된다.
        # early_stopping_rounds : 이 숫자가 가기 전까지 validation score가 증가하지 않으면 round를 멈춘다. 

    # (b) 예측
    pred = pd.Series(model.predict(X_test).round(2))
    return pred, model

In [66]:
# Target 예측

def train_data(X_train, Y_train, X_valid, Y_valid, X_test):

    LGBM_models=[]
    LGBM_actual_pred = pd.DataFrame()

    for q in quantiles:
        print(q)
        pred, model = LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test)
        LGBM_models.append(model)
        LGBM_actual_pred = pd.concat([LGBM_actual_pred,pred],axis=1)

    LGBM_actual_pred.columns=quantiles
    
    return LGBM_models, LGBM_actual_pred

In [67]:
tick = time.time()

# Target1

models_1, results_1 = train_data(X_train_1, Y_train_1, X_valid_1, Y_valid_1, X_test)
results_1.sort_index()[:48]

0.1
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 1.42991
Early stopping, best iteration is:
[270]	valid_0's quantile: 1.42203
0.2
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=0.7 will be ignored. Current value: bagging_fraction=0.6
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 2.25169
[1000]	valid_0's quantile: 2.22587
[1500]	valid_0's quantile: 2.21239
[2000]	valid_0's quantile: 2.20374
[2500]	valid_0's quantile: 2.19931
[3000]	valid_0's quantile: 2.19445
[3500]	valid_0's quantile: 2.18933
[4000]	valid_0's quantile: 2.18481
[4500]	valid_0's quantile: 2.18421
[5000]	valid_0's quantile: 2.18349
[5500]	valid_0's quantile: 2.18323
[6000]	valid_0's quantile: 2.18264
[6500]	valid_0's quantile: 2.1821
[7000]	valid_0's quantile: 2.186
Early stopping, best iteration is:
[6549]	valid_0's quantile: 2.18171
0.3
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=0.7 will be ignored. Current

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [84]:
# Target2

models_2, results_2 = train_data(X_train_2, Y_train_2, X_valid_2, Y_valid_2, X_test)
tock = time.time()
results_2.sort_index()

0.1
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 1.49433
Early stopping, best iteration is:
[177]	valid_0's quantile: 1.48119
0.2
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=0.7 will be ignored. Current value: bagging_fraction=0.6
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 2.4112
[1000]	valid_0's quantile: 2.39722
[1500]	valid_0's quantile: 2.3913
[2000]	valid_0's quantile: 2.3858
[2500]	valid_0's quantile: 2.38168
[3000]	valid_0's quantile: 2.38014
[3500]	valid_0's quantile: 2.3796
[4000]	valid_0's quantile: 2.37785
[4500]	valid_0's quantile: 2.37651
Early stopping, best iteration is:
[4485]	valid_0's quantile: 2.37609
0.3
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=0.7 will be ignored. Current value: bagging_fraction=0.6
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 2.83288
[1000]	valid_0's quantile: 2.78698
[1500]	valid_0's

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
3883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3885,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3886,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
print('소요시간 :', tock-tick)

print(results_1.shape, results_2.shape)

소요시간 : 7321.062851667404
(3888, 9) (3888, 9)


In [86]:
submission

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.csv_Day7_0h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.csv_Day7_0h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.csv_Day7_1h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.csv_Day7_1h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.csv_Day7_2h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7771,80.csv_Day8_21h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7772,80.csv_Day8_22h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7773,80.csv_Day8_22h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7774,80.csv_Day8_23h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [87]:
# 출력

submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = results_1.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = results_2.sort_index().values
submission2 = submission.set_index('id')
submission3 = not_minus(submission2)

In [88]:
submission3.to_csv('submission9.csv')
!cp submission9.csv "drive/My Drive/"